## 1\. Setup the Notebook and verify a connection to PostgreSQL

Modify the connection string below to get your PostgreSQL connection setup for the rest of the notebook.

In [ ]:
import psycopg2 as pg
import pandas.io.sql as psql
import plotly.express as px
import plotly.graph_objects as go
import os

if not os.path.exists("images"):
    os.mkdir("images")

conn = pg.connect(host="localhost", database="postgres", user="postgres", password="password")
df = psql.read_sql('SELECT * from generate_series(1,4)', conn)
df.head()

## Counter with reset

```sql
-- This example resets the counter every 10 rows 
WITH counter_rows AS (
	SELECT ts, 
		CASE WHEN rownum % 10 = 0 THEN 10
		     ELSE rownum % 10 END AS row_counter
	FROM generate_series(now() - INTERVAL '5 minutes', now(), INTERVAL '1 second') WITH ORDINALITY AS t(ts, rownum)
)
SELECT ts, row_counter
FROM counter_rows;
```

In [ ]:
sql_str = """WITH counter_rows AS (
	SELECT ts, 
		CASE WHEN rownum % 10 = 0 THEN 10
		     ELSE rownum % 10 END AS row_counter
	FROM generate_series(now() - INTERVAL '5 minutes', now(), INTERVAL '1 second') WITH ORDINALITY AS t(ts, rownum)
)
SELECT ts, row_counter
FROM counter_rows;
"""
df = psql.read_sql(sql_str, conn)
df

### Daily increasing data (random)

```sql
SELECT ts, (10 + 10 * random()) * rownum as value FROM generate_series
       ( '2020-01-01'::date
       , '2021-12-31'::date
       , INTERVAL '1 day') WITH ORDINALITY AS t(ts, rownum);
```

In [ ]:
graph_title = 'Sample daily website traffic: 2 year'
sql_str = """SELECT ts, (10 + 10 * random()) * rownum as value FROM generate_series
       ( '2020-01-01'::date
       , '2021-12-31'::date
       , INTERVAL '1 day') WITH ORDINALITY AS t(ts, rownum);
"""
df = psql.read_sql(sql_str, conn)

fig = go.Figure([
    go.Scatter(
        name='Measurement',
        x=df['ts'],
        y=df['value'],
        mode='lines',
        line=dict(color='rgb(31, 119, 180)'),
        showlegend=False
    )
])
fig.update_layout(
    yaxis_title='Total website traffic',
    title=graph_title,
    hovermode="x",
    width=900
)
fig.show()
fig.write_image("images/sample_daily_website_traffic-2years.svg")

### 30 day cosine wave (6-day period)

```sql
SELECT  ts,
 cos(rownum-1) as value
FROM generate_series('2021-01-01','2021-01-30',INTERVAL '1 day') WITH ORDINALITY AS t(ts, rownum);
```

In [ ]:
graph_title = 'Basic Cosine Wave (Daily data, 1 Month)'
sql_str = """SELECT  ts,
 cos(rownum-1) as value
FROM generate_series('2021-01-01','2021-01-30',INTERVAL '1 day') WITH ORDINALITY AS t(ts, rownum);
"""
df = psql.read_sql(sql_str, conn)

fig = go.Figure([
    go.Scatter(
        name='Measurement',
        x=df['ts'],
        y=df['value'],
        mode='markers+lines',
        line=dict(color='rgb(31, 119, 180)'),
        showlegend=False
        
    )
])
fig.update_layout(
    yaxis_title='',
    title=graph_title,
    hovermode="x",
    width=900
)
fig.show()
fig.write_image("images/basic_30day_cosine.svg")

#df
# plt.figure(figsize=(15, 8))
# plt.plot(df["time"],df["val"])
# plt.show()

### 90-day cosine wave (6-day period)
```sql
SELECT  ts, 
cos(rownum-1) as value
FROM generate_series('2021-01-01','2021-03-31',INTERVAL '1 day') WITH ORDINALITY AS t(ts, rownum);
```

In [ ]:
graph_title = 'Basic Cosine Wave (Daily data, 3 Months)'
sql_str = """SELECT  ts, 
cos(rownum-1) as value
FROM generate_series('2021-01-01','2021-03-31',INTERVAL '1 day') WITH ORDINALITY AS t(ts, rownum);

"""
df = psql.read_sql(sql_str, conn)

fig = go.Figure([
    go.Scatter(
        name='Measurement',
        x=df['ts'],
        y=df['value'],
        mode='markers+lines',
        line=dict(color='rgb(31, 119, 180)'),
        showlegend=False
        
    )
])
fig.update_layout(
    yaxis_title='',
    title=graph_title,
    hovermode="x",
    width=900
)
fig.show()
fig.write_image("images/basic_90day_cosine.svg")
#df
# plt.figure(figsize=(15, 8))
# plt.plot(df["time"],df["val"])
# plt.show()

### 90-day cosine wave (30-day period)
```sql
SELECT  ts, cos((rownum -1 ) * 6.28/30) as value
FROM generate_series('2021-01-01','2021-03-31',INTERVAL '1 day') WITH ORDINALITY AS t(ts, rownum);
```

In [ ]:
graph_title = 'Cosine wave with 3 periods (Daily values, 3 months)'
sql_str = """SELECT  ts, cos((rownum -1 ) * 6.28/30) as value
FROM generate_series('2021-01-01','2021-03-31',INTERVAL '1 day') WITH ORDINALITY AS t(ts, rownum);
"""
df = psql.read_sql(sql_str, conn)

fig = go.Figure([
    go.Scatter(
        name='Measurement',
        x=df['ts'],
        y=df['value'],
        mode='markers+lines',
        line=dict(color='rgb(31, 119, 180)'),
        showlegend=False
        
    )
])
fig.update_layout(
    yaxis_title='',
    title=graph_title,
    hovermode="x",
    width=900
)
fig.show()
fig.write_image("images/basic_90day_cosine_3_phases.svg")

### 90-day cosine wave (90-day period)
```sql
SELECT  ts, cos((rownum -1 ) * 6.28/90) as value
FROM generate_series('2021-01-01','2021-03-31',INTERVAL '1 day') WITH ORDINALITY AS t(ts, rownum);
```

In [ ]:
graph_title = 'Cosine wave with 1 period (Daily values, 90 days)'
sql_str = """SELECT  ts, cos(rownum * 6.28/90) as value
FROM generate_series('2021-01-01','2021-03-31',INTERVAL '1 day') WITH ORDINALITY AS t(ts, rownum);
"""
df = psql.read_sql(sql_str, conn)

fig = go.Figure([
    go.Scatter(
        name='Measurement',
        x=df['ts'],
        y=df['value'],
        mode='markers+lines',
        line=dict(color='rgb(31, 119, 180)'),
        showlegend=False
        
    )
])
fig.update_layout(
    yaxis_title='',
    title=graph_title,
    hovermode="x",
    width=900
)
fig.show()
fig.write_image("images/basic_90day_cosine_1_phase1.svg")


### 90-day cosine wave (increased min/max)
```sql
SELECT  ts, 10 * cos(rownum * 6.28/30) as value
FROM generate_series('2021-01-01','2021-03-31',INTERVAL '1 day') WITH ORDINALITY AS t(ts, rownum);
```

In [ ]:
graph_title = 'Cosine wave, adjusted amplitude, with 3 periods (Daily values, 3 months)'
sql_str = """SELECT  ts, 10 * cos(rownum * 6.28/30) as value
FROM generate_series('2021-01-01','2021-03-31',INTERVAL '1 day') WITH ORDINALITY AS t(ts, rownum);

"""
df = psql.read_sql(sql_str, conn)

fig = go.Figure([
    go.Scatter(
        name='Measurement',
        x=df['ts'],
        y=df['value'],
        mode='markers+lines',
        line=dict(color='rgb(31, 119, 180)'),
        showlegend=False
    )
])
fig.update_layout(
    yaxis_title='',
    title=graph_title,
    hovermode="x",
    width=900
)
fig.show()
fig.write_image("images/basic_90day_cosine_adjusted_amplitude.svg")

### 90-day cosine wave (shifted min/max)
```sql
SELECT  ts, 10 + 10 * cos(rownum * 6.28/30) as value
FROM generate_series('2021-01-01','2021-03-31',INTERVAL '1 day') WITH ORDINALITY AS t(ts, rownum);
```

In [ ]:
graph_title = 'Cosine wave, Y-axis shift, with 3 periods (Daily values, 3 months)'
sql_str = """SELECT  ts, 10 + 10 * cos(rownum * 6.28/30) as value
FROM generate_series('2021-01-01','2021-03-31',INTERVAL '1 day') WITH ORDINALITY AS t(ts, rownum);
"""
df = psql.read_sql(sql_str, conn)

fig = go.Figure([
    go.Scatter(
        name='Measurement',
        x=df['ts'],
        y=df['value'],
        mode='markers+lines',
        line=dict(color='rgb(31, 119, 180)'),
        showlegend=False
        
    )
])
fig.update_layout(
    yaxis_title='',
    title=graph_title,
    hovermode="x",
    width=900
)
fig.show()
fig.write_image("images/basic_90day_cosine_y_sift.svg")

### Wave-based website visits sample (2 year)
```sql
WITH daily_series AS ( 
	SELECT ts, date(ts) AS day, rownum FROM generate_series
       ( '2020-01-01'
       , '2021-12-31'
       , '1 day'::interval) WITH ORDINALITY AS t(ts, rownum)
),
-- This selects the time, "day", and a 
-- random value that represents our daily website visits
daily_value AS ( 
	SELECT ts, day, rownum, random() AS val
    FROM daily_series
    ORDER BY day
),
-- This cosine wave dataset has the same "day" values which allow 
-- it to be joined to the daily_value easily. The wave value is used to modify
-- the "website" value by some percentage to smooth it out 
-- in the shape of the wave.
daily_wave AS ( 
	SELECT
       day,
       -- 6.28 radians divided by 180 days (rows) to get 
       -- one peak every 6 months (twice a year)
       1 + .2 * cos(rownum * 6.28/180) as p_mod
       FROM daily_series
       day
)
-- (500 + 20 * val) = 500-520 visits per day before modification
-- p_mod = an adjusted cosine value that raises or lowers our data each day
-- row_number = a big incremental value for each row to quickly increase "visits" each day
SELECT dv.ts, (500 + 20 * val) * p_mod * rownum as value
FROM daily_value dv
	INNER JOIN daily_wave dw ON dv.DAY=dw.DAY
    order by ts;
```

In [ ]:
graph_title = 'Wave-based website traffic sample data (2 years)'
sql_str = """WITH daily_series AS ( 
	SELECT ts, date(ts) AS day, rownum FROM generate_series
       ( '2020-01-01'
       , '2021-12-31'
       , '1 day'::interval) WITH ORDINALITY AS t(ts, rownum)
),
-- This selects the time, "day", and a 
-- random value that represents our daily website visits
daily_value AS ( 
	SELECT ts, day, rownum, random() AS val
    FROM daily_series
    ORDER BY day
),
-- This cosine wave dataset has the same "day" values which allow 
-- it to be joined to the daily_value easily. The wave value is used to modify
-- the "website" value by some percentage to smooth it out 
-- in the shape of the wave.
daily_wave AS ( 
	SELECT
       day,
       -- 6.28 radians divided by 180 days (rows) to get 
       -- one peak every 6 months (twice a year)
       1 + .2 * cos(rownum * 6.28/180) as p_mod
       FROM daily_series
       day
)
-- (500 + 20 * val) = 500-520 visits per day before modification
-- p_mod = an adjusted cosine value that raises or lowers our data each day
-- row_number = a big incremental value for each row to quickly increase "visits" each day
SELECT dv.ts, (500 + 20 * val) * p_mod * rownum as value
FROM daily_value dv
	INNER JOIN daily_wave dw ON dv.DAY=dw.DAY
    order by ts;


"""
df = psql.read_sql(sql_str, conn)

fig = go.Figure([
    go.Scatter(
        name='Measurement',
        x=df['ts'],
        y=df['value'],
        mode='lines',
        line=dict(color='rgb(31, 119, 180)'),
        showlegend=False
        
    )
])
fig.update_layout(
    yaxis_title='',
    title=graph_title,
    hovermode="x",
    width=900
)
fig.show()
fig.write_image("images/sample_wave-based_website_traffic-2years.svg")

## Add seasonality with relational data!

**Create a table to represent months of the year**

```sql
CREATE TABLE overrides (
	m_val INT NOT NULL,
	p_inc FLOAT4 NOT null
);
```

**Insert "adjustment" data for various months**
```sql
INSERT INTO overrides(m_val, p_inc) VALUES 
	(1,1.04), -- 4% residual increase from December
	(2,1),
	(3,1),
	(4,1),
	(5,1),
	(6,1.10),-- June increase of 10%
	(7,1),
	(8,1),
	(9,1),
	(10,1),
	(11,1.08), -- 8% early shoppers sales/traffic growth
	(12,1.18); -- 18% holiday increase
```

### Targeted adjustment website visits sample (2 year)
```sql
WITH daily_series AS (
-- a random value that increases over time based on the row number
SELECT ts, date_part('month',ts) AS m_val, (10 + 10*random()) * rownum as value FROM generate_series
       ( '2020-01-01'::date
       , '2021-12-31'::date
       , INTERVAL '1 day') WITH ORDINALITY AS t(ts, rownum)
)
-- join to the `overrides` table to get the 'p_inc' value 
-- for the month of the current row
SELECT ts, value * p_inc AS value FROM daily_series ds
INNER JOIN overrides o ON ds.m_val=o.m_val
ORDER BY ts;
```

In [ ]:
conn = pg.connect(host="localhost", database="lastpoint", user="postgres", password="password")
graph_title = 'Sample website traffic, enhanced with relational data (2 years)'
sql_str = """WITH daily_series AS (
-- a random value that increases over time based on the row number
SELECT ts, date_part('month',ts) AS m_val, (10 + 10*random()) * rownum as value FROM generate_series
       ( '2020-01-01'::date
       , '2021-12-31'::date
       , INTERVAL '1 day') WITH ORDINALITY AS t(ts, rownum)
)
-- join to the `overrides` table to get the 'p_inc' value 
-- for the month of the current row
SELECT ts, value * p_inc AS value FROM daily_series ds
INNER JOIN overrides o ON ds.m_val=o.m_val
ORDER BY ts;
"""
df = psql.read_sql(sql_str, conn)

fig = go.Figure([
    go.Scatter(
        name='Measurement',
        x=df['ts'],
        y=df['value'],
        mode='lines',
        line=dict(color='rgb(31, 119, 180)'),
        showlegend=False
        
    )
])
fig.update_layout(
    yaxis_title='',
    title=graph_title,
    hovermode="x",
    width=1800, 
    height=920
)
fig.update_layout(
    yaxis_title='',
    title=graph_title,
    hovermode="x",
    width=900
)
fig.show()
fig.write_image("images/enhanced_sample_daily_website_traffic-2years.svg")

### Targeted adjustment website visits sample V2 (2 year)
```sql
WITH daily_series AS ( 
	SELECT ts, date(ts) AS day, rownum FROM generate_series
       ( '2020-01-01'
       , '2021-12-31'
       , '1 day'::interval) WITH ORDINALITY AS t(ts, rownum)
),
-- This selects the time, "day", and a 
-- random value that represents our daily website visits
-- 'm_val' will be used to join with the 'overrides' table
daily_value AS ( 
	SELECT ts, day, date_part('month',ts) as m_val, rownum, random() AS val
    FROM daily_series
    ORDER BY day
),
-- This cosine wave dataset has the same "day" values which allow 
-- it to be joined to the daily_value easily. The wave value is used to modify
-- the "website" value by some percentage to smooth it out 
-- in the shape of the wave.
daily_wave AS ( 
	SELECT
       day,
       -- 6.28 radians divided by 180 days (rows) to get 
       -- one peak every 6 months (twice a year)
       1 + .2 * cos(rownum * 6.28/180) as p_mod
       FROM daily_series
       day
)
-- (500 + 20 * val) = 500-520 visits per day before modification
-- p_mod = an adjusted cosine value that raises or lowers our data each day
-- row_number = a big incremental value for each row to quickly increase "visits" each day
-- p_inc = a monthly adjustment value taken from the 'overrides' table
SELECT dv.ts, (500 + 20 * val) * p_mod * rownum * p_inc as value
FROM daily_value dv
	INNER JOIN daily_wave dw ON dv.DAY=dw.DAY
    inner join overrides o on dv.m_val=o.m_val
    order by ts;
```

In [ ]:
conn = pg.connect(host="localhost", database="lastpoint", user="postgres", password="password")
graph_title = 'Wave-based website traffic sample data, enhanced with relational values (2 years)'
sql_str = """-- This is the generate series data
-- with a "short" date to join with later
WITH daily_series AS ( 
	SELECT ts, date(ts) AS day, rownum FROM generate_series
       ( '2020-01-01'
       , '2021-12-31'
       , '1 day'::interval) WITH ORDINALITY AS t(ts, rownum)
),
-- This selects the time, "day", and a 
-- random value that represents our daily website visits
-- 'm_val' will be used to join with the 'overrides' table
daily_value AS ( 
	SELECT ts, day, date_part('month',ts) as m_val, rownum, random() AS val
    FROM daily_series
    ORDER BY day
),
-- This cosine wave dataset has the same "day" values which allow 
-- it to be joined to the daily_value easily. The wave value is used to modify
-- the "website" value by some percentage to smooth it out 
-- in the shape of the wave.
daily_wave AS ( 
	SELECT
       day,
       -- 6.28 radians divided by 180 days (rows) to get 
       -- one peak every 6 months (twice a year)
       1 + .2 * cos(rownum * 6.28/180) as p_mod
       FROM daily_series
       day
)
-- (500 + 20 * val) = 500-520 visits per day before modification
-- p_mod = an adjusted cosine value that raises or lowers our data each day
-- row_number = a big incremental value for each row to quickly increase "visits" each day
-- p_inc = a monthly adjustment value taken from the 'overrides' table
SELECT dv.ts, (500 + 20 * val) * p_mod * rownum * p_inc as value
FROM daily_value dv
	INNER JOIN daily_wave dw ON dv.DAY=dw.DAY
    inner join overrides o on dv.m_val=o.m_val
    order by ts;

"""
df = psql.read_sql(sql_str, conn)

fig = go.Figure([
    go.Scatter(
        name='Measurement',
        x=df['ts'],
        y=df['value'],
        mode='lines',
        line=dict(color='rgb(31, 119, 180)'),
        showlegend=False
        
    )
])
fig.update_layout(
    yaxis_title='',
    title=graph_title,
    hovermode="x",
    width=900
)
fig.show()
fig.write_image("images/sample_wave-based_website_traffic_enhanced-2years.svg")